In [152]:
import pandas as pd
import os
from utils import get_unnoised_labels
from sklearn.linear_model import LogisticRegressionCV, RidgeCV
from sklearn.model_selection import train_test_split

In [153]:
data_dir = ''
csv_path = 'pr_sampled_label_synthetic_numerical_coef_direct1.00.0.csv'
true_effects = pd.read_csv(os.path.join(data_dir, 'label_synthetic_direct_feature_coefs.csv'))

In [154]:
df = pd.read_csv(os.path.join(data_dir, csv_path))
df_nuisance, df = train_test_split(df, test_size=0.8, shuffle=True, random_state=240522)
df_ood, df = train_test_split(df, test_size=0.75, shuffle=True, random_state=240522)

In [ ]:
print(df.shape)
print(df_nuisance.shape)
print(df_ood.shape)

In [ ]:
df.head()

In [ ]:
true_effects

In [158]:
a_name = true_effects.feature[0]
a_true = true_effects.coef[0]
X_nuisance = df_nuisance[true_effects.feature]
A_c_nuisance = X_nuisance.drop(a_name, axis=1)
a_nuisance = df_nuisance[a_name]
y_nuisance = df_nuisance['label_synthetic']

X = df[true_effects.feature]
A_c = X.drop(a_name, axis=1)
X1 = X.copy()
X0 = X.copy()
X1[a_name] = 1
X0[a_name] = 0
a = df[a_name]
y = df['label_synthetic']

X_ood = df_ood[true_effects.feature]
A_c_ood = X_ood.drop(a_name, axis=1)
X1_ood = X_ood.copy()
X0_ood = X_ood.copy()
X1_ood[a_name] = 1
X0_ood[a_name] = 0
a_ood = df_ood[a_name]
y_ood = df_ood['label_synthetic']

In [ ]:
A_c_nuisance

(1) Let $P^*(a^\mathsf{c}(X))=P(a^\mathsf{c}(X)|a(X)=1)$:

In [160]:
clf = LogisticRegressionCV(cv=5)
clf.fit(A_c_nuisance, a_nuisance)

prob_a_cond = clf.predict_proba(A_c)
prob_a_marg = (df[a_name]==1).mean()

prob_a_cond_ood = clf.predict_proba(A_c_ood)
prob_a_marg_ood = (df_ood[a_name]==1).mean()

In [161]:
g = RidgeCV(cv=5)
g.fit(X_nuisance, y_nuisance)
pred_y_ood_1 = g.predict(X1_ood)
pred_y_ood_0 = g.predict(X0_ood)
pred_y_1 = g.predict(X1)
pred_y_0 = g.predict(X0)

In [162]:
# denominator to scale by P(a=1)?
mu1_ipw = ((df[a_name]==1)*df['label_synthetic']).sum()/(df[a_name]==1).sum() 

In [163]:
ipw_0_weight = prob_a_cond[:,1]*(1-prob_a_marg)/(prob_a_cond[:,0]*prob_a_marg)
# denominator to scale by P(a=0)?
mu0_ipw = ((df[a_name]==0)*ipw_0_weight*df['label_synthetic']).sum()/(df[a_name]==0).sum() 

In [164]:
ate_ipw = mu1_ipw - mu0_ipw

In [166]:
mu1_dr1 = ((df[a_name]==1)*(df['label_synthetic'] - pred_y_1)).sum()/(df[a_name]==1).sum() + pred_y_ood_1.mean()

In [167]:
ipw_0_weight_ood = prob_a_cond_ood[:,1]*(1-prob_a_marg_ood)/(prob_a_cond_ood[:,0]*prob_a_marg_ood)
mu0_dr1 = ((df[a_name]==0)*ipw_0_weight*(df['label_synthetic'] - pred_y_0)).sum()/(df[a_name]==0).sum() + \
    ((df_ood[a_name]==0)*ipw_0_weight_ood*pred_y_ood_0).sum()/(df_ood[a_name]==0).sum()

In [170]:
ate_dr1 = mu1_dr1 - mu0_dr1

In [ ]:
print("Estimated '{}' ATE (IPW): {:.3f}".format(a_name, ate_ipw))
print("Estimated '{}' ATE (DR1): {:.3f}".format(a_name, ate_dr1))
print("True '{}' ATE: {:.3f}".format(a_name, a_true))

(2) Let $P^*(a^\mathsf{c}(X))=P(a^\mathsf{c}(X))$: